In [ ]:
import gurobipy as gp
import numpy as np
import matplotlib.pyplot as plt
import random
from random import randint
from sklearn.datasets import make_blobs
from utility import *
import os

Viene importata un'istanza di benchmark e definita la metrica di distanza tra due punti

In [ ]:
dirname = os.path.dirname("__file__")
filename = os.path.join(dirname, 'benchmark/benchmark{8}.txt')
points = load_points(filename)

K = 5
N = len(points)

dist = {}

for i in range(N):
    for j in range(N):
        dist[(i,j)] = np.sum((points[i]-points[j])**2)

Viene creato il modello e vengono definiti le variabili di decisione, la funzione obiettivo e i vincoli del problema sotto esame

In [ ]:
model = gp.Model()
x = model.addVars(N,N, vtype = gp.GRB.BINARY, name = "X") #i elemento , j centroide
y = model.addVars(N, vtype = gp.GRB.BINARY, name = "Y")

obj = model.setObjective(x.prod(dist), gp.GRB.MINIMIZE)

model.addConstrs((sum(x[i,j] for j in range(N)) == 1 for i in range(N)))
model.addConstr(sum(y[j] for j in range(N))==K)
model.addConstrs((x[i,j]<=y[j] for i in range(N) for j in range(N)))

model.optimize()


La soluzione viene convertita nel formato utilizzato dalle euristiche, per chiamare la funzione squared_inner_distance

In [ ]:
centroids = []
for j in range(N):
    if y[j].X == 1:
        centroids.append(j)

sol = np.zeros(N)

for i in range(N):
    for cluster,j in enumerate(centroids):
        if x[i,j].X == 1:
            sol[i] = cluster


print("{}".format(squared_inner_distance(sol, points, K)))